In [ ]:
id_summary = []

for f_name in glob(input_dir + '/*/'):
    if '_unmapped' in f_name:
        continue
    id, ses = f_name.split('_')[0].rsplit('-',1)
    id = id.replace('-', '')
    sub_folder_name = output_dir + 'sub-{}/'.format(id)
    if not os.path.exists(sub_folder_name):
        os.mkdir(sub_folder_name)
    ses_name = 'ses-{}/'.format(ses)
    ses_dir = sub_folder_name + ses_name
    if not os.path.exists(ses_dir):
        os.mkdir(ses_dir)
        session_annotations = annotation[annotation['IDen'] == id]
        
    if not os.path.exists(ses_dir + 'anat'):
        os.mkdir(ses_dir + 'anat')
    file_name = 'sub-{}_ses-{}'.format(id, ses)
    id_summary.append(np.array(['sub-{}'.format(id), 'ses-{}'.format(ses)]))
tsv_df = pd.DataFrame(np.array(id_summary), columns=['participant_id', 'session_id'])
tsv_df.to_csv(output_dir + 'participant_table.tsv', index=False, sep='\t')

In [29]:
folders = glob(input_dir + '/*')

In [3]:
from glob import glob
import os
import pandas as pd
import numpy as np

input_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_niigz/data/'
output_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_bids/'

folders = glob(input_dir + '/*')

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

id_summary = []
file_migration_summary = []
i = 0
for f_name in folders:
    if '_unmapped' in f_name: # skip unmapped folders
        continue

    # get patient id
    id = f_name.split('/')[-1]
    # create patient folders
    pat_folder_name = output_dir + 'sub-{}/'.format(id)
    if not os.path.exists(pat_folder_name):
        os.mkdir(pat_folder_name)

    # get all subdirectories
    sessions = [x[1] for x in os.walk(f_name)][0]
    # get a mapping from orig file location to destination file location
    for sess in sessions:
        temp = [x[2] for x in os.walk(os.path.join(f_name, sess))][0]
        for f in temp:
            orig_path = os.path.join(f_name, sess, f)
            dest_path = os.path.join(pat_folder_name, 'ses-{}/'.format(sess.split('-')[0]), 'anat', f)
            file_migration_summary.append(np.array([id, sess.split('-')[0], sess, orig_path, dest_path]))


    # create subdirectories for each session
    for sess in set([x.split('-')[0] for x in sessions]):
        ses_name = 'ses-{}/'.format(sess)
        sess_folder_name = pat_folder_name + ses_name
        if not os.path.exists(sess_folder_name):
            os.mkdir(sess_folder_name)
        if not os.path.exists(sess_folder_name + 'anat'):
            os.mkdir(sess_folder_name + 'anat')

    file_name = 'sub-{}_ses-{}'.format(id, sess)
    id_summary.append(np.array(['sub-{}'.format(id), 'ses-{}'.format(sess)]))

id_summary_df = pd.DataFrame(np.array(id_summary), columns=['participant_id', 'session_id'])
id_summary_df.to_csv(output_dir + 'participant_table.tsv', index=False, sep='\t')

file_migration_df = pd.DataFrame(np.array(file_migration_summary), columns=['participant_id', 'session_id', 'sess_info', 'orig_path', 'dest_path'])
file_migration_df.to_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/' + 'file_migration_table_v1.tsv', index=False, sep='\t')


In [7]:
from glob import glob
import os

output_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_bids/'
big_folder = glob(output_dir + '/*')

In [2]:
from glob import glob
import os
import pandas as pd
import numpy as np
from shutil import copyfile
from collections import defaultdict


input_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_niigz/data/'
output_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_bids_t1/'

file_summary = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/file_migration_table_t1_v2.tsv', sep='\t')

# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)
# else:
#     raise Exception('The root folder exists.')

id_summary = set()
historical_sess = defaultdict(int)
historical_sess_info = defaultdict(int)

for _, row in file_summary.iterrows():
    # get patient id
    id = row.random_pat_id
    # create patient folders
    pat_folder_name = output_dir + 'sub-{}/'.format(id)
    # if not os.path.exists(pat_folder_name):
    #     os.mkdir(pat_folder_name)

    # create session folder
    sess = int(row['de-identified acc'])
    if row.sess_info not in historical_sess_info:
        historical_sess[sess] += 1
        historical_sess_info[row.sess_info] = historical_sess[sess]
    if historical_sess[sess] > 1:
        sess = str(sess) + '_{}'.format(historical_sess_info[row.sess_info])
    
    ses_name = 'ses-{}/'.format(sess)
    sess_folder_name = pat_folder_name + ses_name
    # if not os.path.exists(sess_folder_name):
    #     os.mkdir(sess_folder_name)
    # if not os.path.exists(sess_folder_name + 'anat'):
    #     os.mkdir(sess_folder_name + 'anat')

    cur_file_name = row.dest_path.replace('.nii.gz', '_T1w.nii.gz').replace('.json', '_T1w.json')
    file_name = 'sub-{}_ses-{}_'.format(id, sess) + cur_file_name
    print(file_name)
    break
    # copy file
    # copyfile(row.orig_path, sess_folder_name + 'anat/' + file_name)

    id_summary.add(('sub-{}'.format(id), 'ses-{}'.format(sess)))

# id_summary_df = pd.DataFrame(np.array(list(id_summary)), columns=['participant_id', 'session_id'])
# id_summary_df.to_csv(output_dir + 'participant_table.tsv', index=False, sep='\t')


sub-5c2f5ed8aeee4c4ba21955ce3b148740_ses-259521128826_20150220-AX_3D_MPR-22_T1w.json


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/file_migration_table_t1_v2.tsv', sep='\t')

In [3]:
df

,sess_info,file_type,de-identified acc,random_pat_id,scan_date_time,label,orig_path,dest_path
0,-20150220-AX_3D_MPR-22,json,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-AX_3D_MPR-22.json
1,-20150220-AX_3D_MPR-22,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-AX_3D_MPR-22.nii.gz
2,-20150220-SAG_MPR_RECON_1MM-23,json,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-SAG_MPR_RECON_1MM-23.json
3,-20150220-SAG_MPR_RECON_1MM-23,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-SAG_MPR_RECON_1MM-23.nii.gz
4,1003422834-20080613-AX_3D_MPR_ISO-13,json,7.862150e+11,644e107e27d540c2a65f159f45fbfe81,2015-08-27 09:56:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20080613-AX_3D_MPR_ISO-13.json
...,...,...,...,...,...,...,...,...
21051,9999070215-20190719-AX_MPR_MPR_sag_recon-18001,niigz,3.160603e+11,16a191ad3fae43ac9be96c18cd86194a,2020-09-16 09:34:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20190719-AX_MPR_MPR_sag_recon-18001.nii.gz
21052,9999070215-20191225-AX_3D_MPR-13,json,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-AX_3D_MPR-13.json
21053,9999070215-20191225-AX_3D_MPR-13,niigz,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-AX_3D_MPR-13.nii.gz
21054,9999070215-20191225-SAG_MPR_RECON-101,json,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-SAG_MPR_RECON-101.json


In [4]:
df = df[df.file_type == 'niigz']

In [6]:
df.to_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/file_migration_table_t1_v4.tsv', sep='\t')